In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import freetype
import cv2

In [ ]:
class Rasterizer(object):
    def __init__(self, in_font:str, gids:set, size:int):
        self.in_font = in_font
        self.gids = gids
        self.size = size
        self.images = {}

    def execute(self, filters=[]):
        self.images = {}
        face = freetype.Face(self.in_font)
        face.set_char_size(self.size*64)
        for gid in self.gids:
            self.images[gid] = self.rasterize_image(face, gid, filters)

        return self.images

    def rasterize_image(self, face, gid, filters=[]):
        W, H = self.size, self.size
        Z = np.zeros( (H, W), dtype=np.ubyte )

        face.load_glyph(gid)
        bitmap = face.glyph.bitmap
        x, y = face.glyph.bitmap_left, face.glyph.bitmap_top
        w, h, p = bitmap.width, bitmap.rows, bitmap.pitch
        shift = -face.descender * self.size // face.units_per_EM
        y += shift

        buff = np.array(bitmap.buffer, dtype=np.ubyte).reshape((h,p))
        Z[H-y:H-y+h,x:x+w].flat |= buff[:,:w].flatten()
        
        for filter in filters:
            Z = filter(Z)

        return Z

In [ ]:
font_path = "xxxxx"

rasterizer = Rasterizer(font_path, {1125,1126,1127,1128,1129,1130}, 224)
images = rasterizer.execute()

iterations = 2
kernel = np.ones((3,3), np.uint8)
gamma = 1.5
alpha = 3.0
beta = 0.0

filters = [
    lambda data: cv2.dilate(data, kernel, iterations=iterations),
    lambda data: cv2.erode(data, kernel, iterations=iterations),
    lambda data: cv2.dilate(data, kernel, iterations=iterations),
    lambda data: cv2.erode(data, kernel, iterations=iterations),
    #lambda data: ((data/255.0)**(gamma)*255).astype(np.ubyte),
    lambda data: np.clip(alpha * data + beta, 0, 255).astype(np.uint8),
]

rasterizer = Rasterizer(font_path, {1125,1126,1127,1128,1129,1130}, 224)
images2 = rasterizer.execute(filters)

In [ ]:
def show_images(images):
    row = 1
    col = len(images)
    plt.figure(figsize=(15,15))

    num = 0

    while num < row * col:
        num += 1
        if num > len(images):
            break
        plt.subplot(row, col, num)
        plt.imshow(images[num-1], cmap = "gray")
        plt.axis('off')

In [ ]:
show_images(list(images.values()))
show_images(list(images2.values()))